<a href="https://colab.research.google.com/github/romit-s/reunion/blob/master/ML_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
df1 = pd.read_csv("loan.csv")


In [ ]:
df2 = pd.read_csv("applicant.csv")
df2.head()
'''Converting Data to int'''
for i in range(len(df2)):
  try:
    df2.at[i,'Has_been_employed_for_at_least']=int(df2.at[i,'Has_been_employed_for_at_least'].split(" ")[0])
  except Exception:
    #substituting null value to mode of the dataframe column
    df2.at[i,'Has_been_employed_for_at_least']=1
  try:
    df2.at[i,'Has_been_employed_for_at_most']=int(df2.at[i,'Has_been_employed_for_at_most'].split(" ")[0])
  except Exception:
    df2.at[i,'Has_been_employed_for_at_most']=4
    
df2.head()

,applicant_id,Primary_applicant_age_in_years,Gender,Marital_status,Number_of_dependents,Housing,Years_at_current_residence,Employment_status,Has_been_employed_for_at_least,Has_been_employed_for_at_most,Telephone,Foreign_worker,Savings_account_balance,Balance_in_existing_bank_account_(lower_limit_of_bucket),Balance_in_existing_bank_account_(upper_limit_of_bucket)
0,1469590,67,male,single,1,own,4,skilled employee / official,7,0,Registered under the applicant's name,1,NaN,NaN,0
1,1203873,22,female,divorced/separated/married,1,own,2,skilled employee / official,1,4,NaN,1,Low,0,2 lac
2,1432761,49,male,single,2,own,3,unskilled - resident,4,7,NaN,1,Low,NaN,NaN
3,1207582,45,male,single,2,for free,4,skilled employee / official,4,7,NaN,1,Low,NaN,0
4,1674436,53,male,single,2,for free,4,skilled employee / official,1,4,NaN,1,Low,NaN,0


Joining the two dataframes

In [ ]:
combined_df = pd.concat([df1.set_index('applicant_id'),df2.set_index('applicant_id')], axis=1, join='inner')
combined_df.columns


Index(['loan_application_id', 'Months_loan_taken_for', 'Purpose',
       'Principal_loan_amount', 'EMI_rate_in_percentage_of_disposable_income',
       'Property', 'Has_coapplicant', 'Has_guarantor', 'Other_EMI_plans',
       'Number_of_existing_loans_at_this_bank', 'Loan_history',
       'high_risk_applicant', 'Primary_applicant_age_in_years', 'Gender',
       'Marital_status', 'Number_of_dependents', 'Housing',
       'Years_at_current_residence', 'Employment_status',
       'Has_been_employed_for_at_least', 'Has_been_employed_for_at_most',
       'Telephone', 'Foreign_worker', 'Savings_account_balance',
       'Balance_in_existing_bank_account_(lower_limit_of_bucket)',
       'Balance_in_existing_bank_account_(upper_limit_of_bucket)'],
      dtype='object')

Dropping Unhelpful columns

In [ ]:
combined_df = combined_df.drop(['Telephone'],axis=1)
combined_df = combined_df.drop(['loan_application_id'],axis=1)
combined_df = combined_df.drop(['Balance_in_existing_bank_account_(lower_limit_of_bucket)'],axis=1) 
combined_df = combined_df.drop(['Balance_in_existing_bank_account_(upper_limit_of_bucket)'],axis=1)
combined_df = combined_df.drop(['Purpose'],axis=1)
combined_df = combined_df.drop(['Other_EMI_plans'],axis=1)
# combined_df = combined_df.drop(['Foreign_worker'],axis=1)
combined_df.isnull().sum()

Months_loan_taken_for                            0
Principal_loan_amount                            0
EMI_rate_in_percentage_of_disposable_income      0
Property                                       154
Has_coapplicant                                  0
Has_guarantor                                    0
Number_of_existing_loans_at_this_bank            0
Loan_history                                     0
high_risk_applicant                              0
Primary_applicant_age_in_years                   0
Gender                                           0
Marital_status                                   0
Number_of_dependents                             0
Housing                                          0
Years_at_current_residence                       0
Employment_status                                0
Has_been_employed_for_at_least                   0
Has_been_employed_for_at_most                    0
Foreign_worker                                   0
Savings_account_balance        

Handling Null Values

In [ ]:
combined_df["Property"].fillna("No property", inplace = True)
combined_df["Savings_account_balance"].fillna("No Balance", inplace=True)
# combined_df['Other_EMI_plans'].fillna('Other_EMI_plans', inplace=True)
combined_df.isnull().sum()

Months_loan_taken_for                          0
Principal_loan_amount                          0
EMI_rate_in_percentage_of_disposable_income    0
Property                                       0
Has_coapplicant                                0
Has_guarantor                                  0
Number_of_existing_loans_at_this_bank          0
Loan_history                                   0
high_risk_applicant                            0
Primary_applicant_age_in_years                 0
Gender                                         0
Marital_status                                 0
Number_of_dependents                           0
Housing                                        0
Years_at_current_residence                     0
Employment_status                              0
Has_been_employed_for_at_least                 0
Has_been_employed_for_at_most                  0
Foreign_worker                                 0
Savings_account_balance                        0
dtype: int64

In [ ]:
y = combined_df['high_risk_applicant']
X = combined_df.drop('high_risk_applicant', axis=1)
combined_df.columns.get_loc("Housing")

13

One Hot Encoding Categorical variables

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
transformer = make_column_transformer((OneHotEncoder(), ['Housing','Marital_status' ,'Property' , 'Loan_history','Gender','Employment_status','Savings_account_balance']),remainder='passthrough')

transformed = transformer.fit_transform(X)
transformed_df = pd.DataFrame(transformed, columns=transformer.get_feature_names())
X = np.array(transformed_df)
X

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=.2, random_state=123)

In [ ]:
print(X_train)

[[0.0 0.0 1.0 ... 4 7 1]
 [1.0 0.0 0.0 ... 1 4 1]
 [0.0 1.0 0.0 ... 7 0 1]
 ...
 [0.0 0.0 1.0 ... 4 7 1]
 [0.0 1.0 0.0 ... 4 7 1]
 [0.0 1.0 0.0 ... 4 7 1]]


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Artifical Neural Network Model

In [ ]:
import tensorflow as tf
ann = tf.keras.models.Sequential()

In [ ]:

ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
25/25 [==============================] - 1s 2ms/step - loss: 0.7197 - accuracy: 0.4775
Epoch 2/100
25/25 [==============================] - 0s 2ms/step - loss: 0.6855 - accuracy: 0.5825
Epoch 3/100
25/25 [==============================] - 0s 2ms/step - loss: 0.6631 - accuracy: 0.6325
Epoch 4/100
25/25 [==============================] - 0s 2ms/step - loss: 0.6472 - accuracy: 0.6725
Epoch 5/100
25/25 [==============================] - 0s 3ms/step - loss: 0.6332 - accuracy: 0.7000
Epoch 6/100
25/25 [==============================] - 0s 3ms/step - loss: 0.6207 - accuracy: 0.7125
Epoch 7/100
25/25 [==============================] - 0s 3ms/step - loss: 0.6088 - accuracy: 0.7250
Epoch 8/100
25/25 [==============================] - 0s 3ms/step - loss: 0.5979 - accuracy: 0.7300
Epoch 9/100
25/25 [==============================] - 0s 3ms/step - loss: 0.5885 - accuracy: 0.7312
Epoch 10/100
25/25 [==============================] - 0s 3ms/step - loss: 0.5792 - accuracy: 0.7312
Epoch 11/

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.3)
# print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

7/7 [==============================] - 0s 4ms/step


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[104  35]
 [ 25  36]]


0.7

Naive Bayes Model

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[107  32]
 [ 24  37]]


0.72

Kernel SVC Model

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[133   6]
 [ 48  13]]


0.73